Notebook dedicado a explorar métodos de lectura y escritura de un DF así como el particionado de los datos. Para ello se trabajara con un fichero del portal de datos abiertos del ayuntamiento de Madrid, especificamente el Padron de Madrid, fichero CSV el cual es necesario realizar un tratamiento previo sobre los datos.

En este Notebook leeremos datos en formato CSV y se almacena en los formatos (para posteriormente hacer pruebas):

CSV
JSON
Parquet
Parquet partitionado

In [1]:
import org.apache.spark.sql.types.{StringType, StructField, IntegerType, StructType}

val file_location = "../data/Rango_Edades_Seccion_202012.csv"

import org.apache.spark.sql.Row

// Se define el esquema de los datos
val schema = StructType(
  Array(
    StructField("COD_DISTRITO", IntegerType, true),
    StructField("DESC_DISTRITO", StringType, true),
    StructField("COD_DIST_BARRIO", IntegerType, true),
    StructField("DESC_BARRIO", StringType, true),
    StructField("COD_BARRIO", IntegerType, true),
    StructField("COD_DIST_SECCION", IntegerType, true),
    StructField("COD_SECCION", IntegerType, true),
    StructField("COD_EDAD_INT", IntegerType, true),
    StructField("EspanolesHombres", IntegerType, true),
    StructField("EspanolesMujeres", IntegerType, true),
    StructField("ExtranjerosHombres", IntegerType, true),
    StructField("ExtranjerosMujeres", IntegerType, true)
   )
)

// Leemos los datos desde un CV indicando que:
// El fichero CSV incluye cabecera
// No se inferirá el esquema y nosotros indicaremos el esquema adacuado
// Los valores vacios se reemplazaran por 0
// Se indica que el separador de columna es el ;
// Se indica que el caracter para escapar valores es "
// Se indica que los valores de cada columna/campo estará encerrado en "
// Se modifica el encoding para que tolere valores como ñ, ç, á, por mecionar algunos
// Aunque no surge ningún efecto, se indica que se haga trim a los datos por derecha e izquierda
val sourceDF = spark.read.format("csv")
.option("inferSchema", false)
.option("header", true)
.option("emptyValue", "0")
.option("ignoreLeadingWhiteSpace", true)
.option("ignoreTrailingWhiteSpace", true)
.option("sep", ";")
.option("quote", "\"")
.option("escape", "\"")
.option("encoding", "ISO-8859-1")
.schema(schema)
.load(file_location)

sourceDF.show()

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.36:4044
SparkContext available as 'sc' (version = 3.0.2, master = local[*], app id = local-1615757653988)
SparkSession available as 'spark'


+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|       DESC_DISTRITO|COD_DIST_BARRIO|         DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+--------------------+---------------+--------------------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|           1|CENTRO              |            101|PALACIO             |         1|            1006|          6|         103|               0|               1|                 0|                 0|
|           1|CENTRO              |            101|PALACIO             |         1|            1007|          7|           0|               1|               1|                 0|                 3|
|         

import org.apache.spark.sql.types.{StringType, StructField, IntegerType, StructType}
file_location: String = ../data/Rango_Edades_Seccion_202012.csv
import org.apache.spark.sql.Row
schema: org.apache.spark.sql.types.StructType = StructType(StructField(COD_DISTRITO,IntegerType,true), StructField(DESC_DISTRITO,StringType,true), StructField(COD_DIST_BARRIO,IntegerType,true), StructField(DESC_BARRIO,StringType,true), StructField(COD_BARRIO,IntegerType,true), StructField(COD_DIST_SECCION,IntegerType,true), StructField(COD_SECCION,IntegerType,true), StructField(COD_EDAD_INT,IntegerType,true), StructField(EspanolesHombres,IntegerType,true), StructField(EspanolesMujeres,IntegerType,true), StructField(ExtranjerosHombres,IntegerType,true), StructField(ExtranjerosMujeres,IntegerType,true))
sourceD...


In [2]:
import org.apache.spark.sql.functions.{col, trim}

// Se invoca la función trim únicamente en aquellos campos del tipo String
val sourceDFWithTrimedValues = sourceDF.select(schema.fields.map(field => {
  if (field.dataType == StringType) {
   trim(col(field.name)).as(field.name) 
  } else {
    col(field.name)
  }
}):_*).cache()

sourceDFWithTrimedValues.show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|           1|       CENTRO|            101|    PALACIO|         1|            1006|          6|         103|               0|               1|                 0|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1007|          7|           0|               1|               1|                 0|                 3|
|           1|       CENTRO|            101|    PALACIO|         1|            1007|      

import org.apache.spark.sql.functions.{col, trim}
sourceDFWithTrimedValues: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


In [3]:
// comprobamos filtrando por aquellos registros cuyo nombre de barrio es Palacio sin incluir espacios en blanco para comprobar que el trim ha funcionado
sourceDFWithTrimedValues.where(col("DESC_BARRIO") === "PALACIO").show()

+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|COD_DISTRITO|DESC_DISTRITO|COD_DIST_BARRIO|DESC_BARRIO|COD_BARRIO|COD_DIST_SECCION|COD_SECCION|COD_EDAD_INT|EspanolesHombres|EspanolesMujeres|ExtranjerosHombres|ExtranjerosMujeres|
+------------+-------------+---------------+-----------+----------+----------------+-----------+------------+----------------+----------------+------------------+------------------+
|           1|       CENTRO|            101|    PALACIO|         1|            1006|          6|         103|               0|               1|                 0|                 0|
|           1|       CENTRO|            101|    PALACIO|         1|            1007|          7|           0|               1|               1|                 0|                 3|
|           1|       CENTRO|            101|    PALACIO|         1|            1007|      

In [4]:
import org.apache.spark.sql.functions.asc

sourceDFWithTrimedValues.select("DESC_DISTRITO", "DESC_BARRIO")
                        .distinct()
                        .orderBy(asc("DESC_DISTRITO"), asc("DESC_BARRIO"))
                        .show()

+-------------+----------------+
|DESC_DISTRITO|     DESC_BARRIO|
+-------------+----------------+
|   ARGANZUELA|         ACACIAS|
|   ARGANZUELA|          ATOCHA|
|   ARGANZUELA|         CHOPERA|
|   ARGANZUELA|        DELICIAS|
|   ARGANZUELA|        IMPERIAL|
|   ARGANZUELA|         LEGAZPI|
|   ARGANZUELA| PALOS DE MOGUER|
|      BARAJAS|      AEROPUERTO|
|      BARAJAS|ALAMEDA DE OSUNA|
|      BARAJAS| CASCO H.BARAJAS|
|      BARAJAS|      CORRALEJOS|
|      BARAJAS|           TIMON|
|  CARABANCHEL|        ABRANTES|
|  CARABANCHEL|      BUENAVISTA|
|  CARABANCHEL|        COMILLAS|
|  CARABANCHEL|          OPAÑEL|
|  CARABANCHEL|   PUERTA BONITA|
|  CARABANCHEL|      SAN ISIDRO|
|  CARABANCHEL|    VISTA ALEGRE|
|       CENTRO|          CORTES|
+-------------+----------------+
only showing top 20 rows



import org.apache.spark.sql.functions.asc


In [5]:
// Se guardan los datos como una tabla en formato parquet
sourceDFWithTrimedValues.write.mode("overwrite").saveAsTable("padron_madrid")
// Se guardan los datos como una tabla particionada por los campos DESC_DISTRITO y DESC_BARRIO en formato parquet 
sourceDFWithTrimedValues.write.mode("overwrite")
    .partitionBy("DESC_DISTRITO", "DESC_BARRIO")
    .saveAsTable("padron_madrid_particionado")

// reducimos el numero de partitiones para que de esta forma sea un único fichero JSON o CSV el que sea generado al llevar a cabo la escritura
val onePartitionDF = sourceDFWithTrimedValues.coalesce(1)

// Se guardan los datos como un fichero CSV con cabecera
onePartitionDF.write.mode("overwrite").format("csv").option("header", true).save("./dataOutput/df/csv")
// Se guardan los datos como un fichero JSON
onePartitionDF.write.mode("overwrite").json("./dataOutput/df/json")
// Se guardan los datos como un fichero Parquet
onePartitionDF.write.mode("overwrite").parquet("./dataOutput/df/parquet")

onePartitionDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COD_DISTRITO: int, DESC_DISTRITO: string ... 10 more fields]


In [6]:
spark.sql("SHOW CREATE TABLE padron_madrid_particionado").show(false)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                                                                                                                                                                                                                        |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
import scala.sys.process._

// Lista el contenido de la carpeta /dataOutput/df/csv y el tamaño de los ficheros en un formato leible
"ls -lha ./dataOutput/df/csv"!!

import scala.sys.process._
res6: String =
"total 24456
drwxr-xr-x  6 joseantoniorodriguez  staff   192B 14 mar 22:34 .
drwxr-xr-x  5 joseantoniorodriguez  staff   160B 14 mar 22:34 ..
-rw-r--r--  1 joseantoniorodriguez  staff     8B 14 mar 22:34 ._SUCCESS.crc
-rw-r--r--  1 joseantoniorodriguez  staff    95K 14 mar 22:34 .part-00000-f2de7c57-86d4-4cc7-a203-bd35fed72cec-c000.csv.crc
-rw-r--r--  1 joseantoniorodriguez  staff     0B 14 mar 22:34 _SUCCESS
-rw-r--r--  1 joseantoniorodriguez  staff    12M 14 mar 22:34 part-00000-f2de7c57-86d4-4cc7-a203-bd35fed72cec-c000.csv
"


In [8]:
"ls -lha ./dataOutput/df/json"!!

res7: String =
"total 122696
drwxr-xr-x  6 joseantoniorodriguez  staff   192B 14 mar 22:34 .
drwxr-xr-x  5 joseantoniorodriguez  staff   160B 14 mar 22:34 ..
-rw-r--r--  1 joseantoniorodriguez  staff     8B 14 mar 22:34 ._SUCCESS.crc
-rw-r--r--  1 joseantoniorodriguez  staff   476K 14 mar 22:34 .part-00000-97b42ed8-af72-4dce-bee0-32553eba7af3-c000.json.crc
-rw-r--r--  1 joseantoniorodriguez  staff     0B 14 mar 22:34 _SUCCESS
-rw-r--r--  1 joseantoniorodriguez  staff    59M 14 mar 22:34 part-00000-97b42ed8-af72-4dce-bee0-32553eba7af3-c000.json
"


In [9]:
"ls -lha ./dataOutput/df/parquet"!!

res8: String =
"total 1376
drwxr-xr-x  6 joseantoniorodriguez  staff   192B 14 mar 22:34 .
drwxr-xr-x  5 joseantoniorodriguez  staff   160B 14 mar 22:34 ..
-rw-r--r--  1 joseantoniorodriguez  staff     8B 14 mar 22:34 ._SUCCESS.crc
-rw-r--r--  1 joseantoniorodriguez  staff   5,3K 14 mar 22:34 .part-00000-073dbfb9-b744-4e7e-86ac-51159deeabac-c000.snappy.parquet.crc
-rw-r--r--  1 joseantoniorodriguez  staff     0B 14 mar 22:34 _SUCCESS
-rw-r--r--  1 joseantoniorodriguez  staff   674K 14 mar 22:34 part-00000-073dbfb9-b744-4e7e-86ac-51159deeabac-c000.snappy.parquet
"
